In [2]:
!pip install beautifulsoup4
!pip install lxml
!pip install geopy
!pip install geocoder
import numpy as np
import pandas as pd
import json 
from geopy.geocoders import Nominatim 
import geocoder 
import requests 
from bs4 import BeautifulSoup 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 
import urllib.request
print("Libraries imported.")

Libraries imported.


In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [4]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [5]:
neighborhoods_data = newyork_data['features']

In [6]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

Transforming the data into a pandas Dataframe

In [7]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
neighborhoods = pd.DataFrame(columns=column_names)

In [ ]:
neighborhoods

In [8]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [9]:
neighborhoods.tail()

,Borough,Neighborhood,Latitude,Longitude
301,Manhattan,Hudson Yards,40.756658,-74.000111
302,Queens,Hammels,40.587338,-73.805530
303,Queens,Bayswater,40.611322,-73.765968
304,Queens,Queensbridge,40.756091,-73.945631
305,Staten Island,Fox Hills,40.617311,-74.081740


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]))

The dataframe has 5 boroughs and 306 neighborhoods.


In [11]:
address = 'New York City, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [12]:
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Our Restaurant should be in the bronx, so we make a dataframe consisting only of party in the Bronx

In [13]:
bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
bronx_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [14]:
address = 'Bronx, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Bronx are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Bronx are 40.8466508, -73.8785937.


Now we make a visualization of just the Bronx

In [15]:
map_bronx= folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(bronx_data['Latitude'], bronx_data['Longitude'], bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_bronx)  
    
map_bronx

To explore the Bronx, a definition of the Foursquare credentials is necessary

In [16]:
CLIENT_ID = '4BNCWXHSESMHVRWW5O2PEJSUXFUGLIJI0GWAOFHSB4R1VPCK' # your Foursquare ID
CLIENT_SECRET = 'SPG2HLXJECWQZARCDUZHORNDUK5ZYNEDYBHWFB0ZPOU11STA' # your Foursquare Secret
VERSION = '20180605'

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: 4BNCWXHSESMHVRWW5O2PEJSUXFUGLIJI0GWAOFHSB4R1VPCK
CLIENT_SECRET:SPG2HLXJECWQZARCDUZHORNDUK5ZYNEDYBHWFB0ZPOU11STA


In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [18]:
bronx_venues = getNearbyVenues(names=bronx_data['Neighborhood'],
                                   latitudes=bronx_data['Latitude'],
                                   longitudes=bronx_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights


In [19]:
print(bronx_venues.shape)
bronx_venues.head()

(1216, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,40.894123,-73.845892,Dessert Shop
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,40.890487,-73.848568,Ice Cream Shop
2,Wakefield,40.894705,-73.847201,Walgreens,40.896528,-73.844700,Pharmacy
3,Wakefield,40.894705,-73.847201,Rite Aid,40.896649,-73.844846,Pharmacy
4,Wakefield,40.894705,-73.847201,Dunkin',40.890459,-73.849089,Donut Shop


We clean the data, so that we only have all the columns and rows left which are interesting for us. We want first focus on the coordinates of the venues and second focus on Restaurants.

In [20]:
bronx_venues.drop(["Neighborhood Latitude", "Neighborhood Longitude"],axis=1,inplace=True)
bronx_venues.tail()

,Neighborhood,Venue,Venue Latitude,Venue Longitude,Venue Category
1211,Kingsbridge Heights,New Yung Hong Chinese Restaurant,40.873272,-73.905128,Chinese Restaurant
1212,Kingsbridge Heights,Caridad Spanish Restaurant,40.871832,-73.906700,Spanish Restaurant
1213,Kingsbridge Heights,King's Farm,40.867376,-73.897498,Restaurant
1214,Kingsbridge Heights,MTA Subway - Kingsbridge Rd (4),40.867637,-73.897184,Metro Station
1215,Kingsbridge Heights,Lehman College Ballfield,40.871007,-73.895781,Baseball Field


In [21]:
bronx_venues.rename(columns={"Venue Category" : "Category"}, 
                 inplace=True)

In [22]:
bronx_venues[bronx_venues.Category.str.contains('Restaurant',case=False)]

,Neighborhood,Venue,Venue Latitude,Venue Longitude,Category
16,Co-op City,Arby's,40.870411,-73.828606,Fast Food Restaurant
17,Co-op City,Townhouse Restaurant,40.876086,-73.828868,Restaurant
20,Co-op City,Guang Hui Chinese Restaurant,40.876651,-73.829092,Chinese Restaurant
24,Eastchester,Fish & Ting,40.885656,-73.829197,Caribbean Restaurant
28,Eastchester,Dyre Fish Market,40.889318,-73.831453,Seafood Restaurant
...,...,...,...,...,...
1202,Kingsbridge Heights,La Catrina,40.869960,-73.903956,Mexican Restaurant
1206,Kingsbridge Heights,New No. 1 Chinese Restaurnt,40.868266,-73.899965,Chinese Restaurant
1211,Kingsbridge Heights,New Yung Hong Chinese Restaurant,40.873272,-73.905128,Chinese Restaurant
1212,Kingsbridge Heights,Caridad Spanish Restaurant,40.871832,-73.906700,Spanish Restaurant


Now lets just take all the cinemas in the bronx

In [23]:
bronx_venues = bronx_venues[bronx_venues.Category.str.contains('Restaurant',case=False)]

In [26]:
bronx_venues.head()

,index,Neighborhood,Venue,Venue Latitude,Venue Longitude,Category
0,16,Co-op City,Arby's,40.870411,-73.828606,Fast Food Restaurant
1,17,Co-op City,Townhouse Restaurant,40.876086,-73.828868,Restaurant
2,20,Co-op City,Guang Hui Chinese Restaurant,40.876651,-73.829092,Chinese Restaurant
3,24,Eastchester,Fish & Ting,40.885656,-73.829197,Caribbean Restaurant
4,28,Eastchester,Dyre Fish Market,40.889318,-73.831453,Seafood Restaurant


In [25]:
bronx_venues = bronx_venues.reset_index(level=None, drop=False, inplace=False, col_level=0, col_fill='')
bronx_venues.head()

,index,Neighborhood,Venue,Venue Latitude,Venue Longitude,Category
0,16,Co-op City,Arby's,40.870411,-73.828606,Fast Food Restaurant
1,17,Co-op City,Townhouse Restaurant,40.876086,-73.828868,Restaurant
2,20,Co-op City,Guang Hui Chinese Restaurant,40.876651,-73.829092,Chinese Restaurant
3,24,Eastchester,Fish & Ting,40.885656,-73.829197,Caribbean Restaurant
4,28,Eastchester,Dyre Fish Market,40.889318,-73.831453,Seafood Restaurant


In [27]:
bronx_venues.tail()

,index,Neighborhood,Venue,Venue Latitude,Venue Longitude,Category
251,1202,Kingsbridge Heights,La Catrina,40.869960,-73.903956,Mexican Restaurant
252,1206,Kingsbridge Heights,New No. 1 Chinese Restaurnt,40.868266,-73.899965,Chinese Restaurant
253,1211,Kingsbridge Heights,New Yung Hong Chinese Restaurant,40.873272,-73.905128,Chinese Restaurant
254,1212,Kingsbridge Heights,Caridad Spanish Restaurant,40.871832,-73.906700,Spanish Restaurant
255,1213,Kingsbridge Heights,King's Farm,40.867376,-73.897498,Restaurant


In [28]:
print('There are {} uniques categories.'.format(len(bronx_venues['Category'].unique())))

There are 30 uniques categories.


We can list what kind of restaurant these are.

In [29]:
bronx_venues.Category.unique()

array(['Fast Food Restaurant', 'Restaurant', 'Chinese Restaurant',
       'Caribbean Restaurant', 'Seafood Restaurant',
       'Latin American Restaurant', 'Spanish Restaurant',
       'Mexican Restaurant', 'Vegetarian / Vegan Restaurant',
       'Italian Restaurant', 'Indian Restaurant', 'American Restaurant',
       'Sushi Restaurant', 'French Restaurant', 'African Restaurant',
       'Greek Restaurant', 'Paella Restaurant', 'Asian Restaurant',
       'Peruvian Restaurant', 'South American Restaurant',
       'South Indian Restaurant', 'Southern / Soul Food Restaurant',
       'Middle Eastern Restaurant', 'Arepa Restaurant',
       'Eastern European Restaurant', 'Japanese Restaurant',
       'Mediterranean Restaurant', 'Thai Restaurant',
       'Comfort Food Restaurant', 'Vietnamese Restaurant'], dtype=object)

Now we will also count how often each of the kind is represented

In [31]:
bronx_venues['Category'].value_counts()

Chinese Restaurant                 41
Italian Restaurant                 40
Fast Food Restaurant               28
Spanish Restaurant                 28
Mexican Restaurant                 25
Latin American Restaurant          20
Caribbean Restaurant               14
American Restaurant                13
Asian Restaurant                    7
Restaurant                          7
Seafood Restaurant                  6
Indian Restaurant                   3
Peruvian Restaurant                 2
African Restaurant                  2
Eastern European Restaurant         2
Sushi Restaurant                    2
Japanese Restaurant                 2
Thai Restaurant                     2
Southern / Soul Food Restaurant     1
Vietnamese Restaurant               1
South Indian Restaurant             1
South American Restaurant           1
Mediterranean Restaurant            1
Middle Eastern Restaurant           1
Greek Restaurant                    1
Paella Restaurant                   1
Arepa Restau

In [32]:
map_restaurants= folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(bronx_venues['Venue Latitude'], bronx_venues['Venue Longitude'], bronx_venues['Venue']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_restaurants)  
    
map_restaurants

In [33]:
bronx_venues.Category.unique()

array(['Fast Food Restaurant', 'Restaurant', 'Chinese Restaurant',
       'Caribbean Restaurant', 'Seafood Restaurant',
       'Latin American Restaurant', 'Spanish Restaurant',
       'Mexican Restaurant', 'Vegetarian / Vegan Restaurant',
       'Italian Restaurant', 'Indian Restaurant', 'American Restaurant',
       'Sushi Restaurant', 'French Restaurant', 'African Restaurant',
       'Greek Restaurant', 'Paella Restaurant', 'Asian Restaurant',
       'Peruvian Restaurant', 'South American Restaurant',
       'South Indian Restaurant', 'Southern / Soul Food Restaurant',
       'Middle Eastern Restaurant', 'Arepa Restaurant',
       'Eastern European Restaurant', 'Japanese Restaurant',
       'Mediterranean Restaurant', 'Thai Restaurant',
       'Comfort Food Restaurant', 'Vietnamese Restaurant'], dtype=object)

In [34]:
bronx_venues.Category.count()

256

## Analyze Each Neighborhood and its Restaurants

In [35]:
bronx_onehot = pd.get_dummies(bronx_venues[['Neighborhood']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bronx_onehot['Category'] = bronx_venues['Category'] 

# move neighborhood column to the first column
fixed_columns = [bronx_onehot.columns[-1]] + list(bronx_onehot.columns[:-1])
bronx_onehot = bronx_onehot[fixed_columns]


bronx_onehot.head()

,Category,Allerton,Baychester,Bedford Park,Belmont,Bronxdale,City Island,Claremont Village,Clason Point,Co-op City,...,Soundview,Spuyten Duyvil,Throgs Neck,Unionport,University Heights,Van Nest,West Farms,Westchester Square,Williamsbridge,Woodlawn
0,Fast Food Restaurant,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Restaurant,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Chinese Restaurant,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Caribbean Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Seafood Restaurant,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
bronx_onehot.shape

(256, 47)

## Now we group by each Restaurant category

In [37]:
bronx_grouped = bronx_onehot.groupby('Category').mean().reset_index()
bronx_grouped

,Category,Allerton,Baychester,Bedford Park,Belmont,Bronxdale,City Island,Claremont Village,Clason Point,Co-op City,...,Soundview,Spuyten Duyvil,Throgs Neck,Unionport,University Heights,Van Nest,West Farms,Westchester Square,Williamsbridge,Woodlawn
0,African Restaurant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.500000,0.000000,0.00,0.000000,0.000000,0.000000
1,American Restaurant,0.076923,0.000000,0.000000,0.076923,0.000000,0.076923,0.000000,0.0,0.000000,...,0.000000,0.0,0.076923,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.076923
2,Arepa Restaurant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
3,Asian Restaurant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.142857,0.000000,0.000000,0.000000,0.00,0.142857,0.000000,0.000000
4,Caribbean Restaurant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.071429,0.0,0.000000,...,0.000000,0.0,0.000000,0.071429,0.000000,0.000000,0.00,0.000000,0.142857,0.000000
5,Chinese Restaurant,0.048780,0.000000,0.097561,0.024390,0.024390,0.000000,0.048780,0.0,0.024390,...,0.073171,0.0,0.000000,0.024390,0.073171,0.000000,0.00,0.024390,0.000000,0.000000
6,Comfort Food Restaurant,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
7,Eastern European Restaurant,0.000000,0.000000,0.000000,0.500000,0.500000,0.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
8,Fast Food Restaurant,0.035714,0.035714,0.000000,0.035714,0.000000,0.000000,0.000000,0.0,0.035714,...,0.000000,0.0,0.000000,0.035714,0.035714,0.000000,0.00,0.107143,0.000000,0.000000
9,French Restaurant,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000


In [38]:
bronx_grouped.shape

(30, 47)

## Let's print each Restaurant along with the Top districts where they appear

In [39]:
num_top_venues = 5

for hood in bronx_grouped['Category']:
    print("----"+hood+"----")
    temp = bronx_grouped[bronx_grouped['Category'] == hood].T.reset_index()
    temp.columns = ['Neighborhood','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----African Restaurant----
         Neighborhood  freq
0  University Heights   0.5
1             Fordham   0.5
2            Allerton   0.0
3         Port Morris   0.0
4          Mount Hope   0.0


----American Restaurant----
     Neighborhood  freq
0   Schuylerville  0.15
1     Parkchester  0.15
2        Allerton  0.08
3  Edgewater Park  0.08
4     Throgs Neck  0.08


----Arepa Restaurant----
      Neighborhood  freq
0      Morris Park   1.0
1      Port Morris   0.0
2       Mount Eden   0.0
3       Mount Hope   0.0
4  North Riverdale   0.0


----Asian Restaurant----
         Neighborhood  freq
0  Westchester Square  0.14
1         Parkchester  0.14
2         Throgs Neck  0.14
3          Pelham Bay  0.14
4       Schuylerville  0.14


----Caribbean Restaurant----
     Neighborhood  freq
0     Eastchester  0.21
1  Williamsbridge  0.14
2       Olinville  0.14
3         Norwood  0.07
4     Parkchester  0.07


----Chinese Restaurant----
          Neighborhood  freq
0         Bedford Park  0.

## Transform that into a pandas dataframe

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Category']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Neighborhood'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Neighborhood'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Category'] = bronx_grouped['Category']

for ind in np.arange(bronx_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bronx_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Category,1st Most Common Neighborhood,2nd Most Common Neighborhood,3rd Most Common Neighborhood,4th Most Common Neighborhood,5th Most Common Neighborhood,6th Most Common Neighborhood,7th Most Common Neighborhood,8th Most Common Neighborhood,9th Most Common Neighborhood,10th Most Common Neighborhood
0,African Restaurant,University Heights,Fordham,Woodlawn,Concourse Village,Longwood,Kingsbridge Heights,Kingsbridge,Hunts Point,High Bridge,Edgewater Park
1,American Restaurant,Schuylerville,Parkchester,Woodlawn,Belmont,City Island,Edgewater Park,Fordham,Norwood,Pelham Gardens,Allerton
2,Arepa Restaurant,Morris Park,East Tremont,Melrose,Longwood,Kingsbridge Heights,Kingsbridge,Hunts Point,High Bridge,Fordham,Edgewater Park
3,Asian Restaurant,Pelham Bay,Westchester Square,High Bridge,Throgs Neck,Edgewater Park,Schuylerville,Parkchester,Woodlawn,Kingsbridge,Hunts Point
4,Caribbean Restaurant,Eastchester,Williamsbridge,Olinville,Kingsbridge,Unionport,Fordham,Claremont Village,Norwood,Concourse,Parkchester


## Clustering Restaurants

In [53]:
kclusters = 29

bronx_grouped_clustering = bronx_grouped.drop('Category', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bronx_grouped_clustering)

kmeans.labels_[0:29] 

array([18, 25,  6, 20, 23, 24,  7, 16, 26,  2,  8, 17,  0, 22,  5, 19, 27,
       10,  3, 15,  1, 21,  9,  4,  4, 28, 13, 14, 12], dtype=int32)

In [58]:
neighborhoods_venues_sorted2.insert(0, 'Cluster Labels', kmeans.labels_)



NameError: name 'neighborhoods_venues_sorted2' is not defined

In [59]:
bronx_venues_categroy.head()

,index,Neighborhood,Venue,Venue Latitude,Venue Longitude,Category,Cluster Labels,1st Most Common Neighborhood,2nd Most Common Neighborhood,3rd Most Common Neighborhood,4th Most Common Neighborhood,5th Most Common Neighborhood,6th Most Common Neighborhood,7th Most Common Neighborhood,8th Most Common Neighborhood,9th Most Common Neighborhood,10th Most Common Neighborhood
0,16,Co-op City,Arby's,40.870411,-73.828606,Fast Food Restaurant,2,Westchester Square,Fordham,Mount Eden,Kingsbridge,Morrisania,Concourse Village,Pelham Bay,Allerton,Eastchester,East Tremont
1,17,Co-op City,Townhouse Restaurant,40.876086,-73.828868,Restaurant,2,Kingsbridge Heights,Van Nest,Hunts Point,Norwood,Co-op City,Port Morris,Parkchester,Woodlawn,East Tremont,Kingsbridge
2,20,Co-op City,Guang Hui Chinese Restaurant,40.876651,-73.829092,Chinese Restaurant,2,Bedford Park,Soundview,University Heights,Kingsbridge Heights,Pelham Parkway,Claremont Village,Pelham Gardens,Allerton,Norwood,Melrose
3,24,Eastchester,Fish & Ting,40.885656,-73.829197,Caribbean Restaurant,2,Eastchester,Williamsbridge,Olinville,Kingsbridge,Unionport,Fordham,Claremont Village,Norwood,Concourse,Parkchester
4,28,Eastchester,Dyre Fish Market,40.889318,-73.831453,Seafood Restaurant,2,City Island,Morrisania,Kingsbridge,Unionport,Eastchester,Woodlawn,East Tremont,Longwood,Kingsbridge Heights,Hunts Point


In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(bronx_venues_categroy['Venue Latitude'], bronx_venues_categroy['Venue Longitude'], bronx_venues_categroy['Category'], bronx_venues_categroy['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine the Clusters

In [60]:
bronx_venues_categroy.loc[bronx_venues_categroy['Cluster Labels'] == 0, bronx_venues_categroy.columns[[1] + list(range(5, bronx_venues_categroy.shape[1]))]]

,Neighborhood,Category,Cluster Labels,1st Most Common Neighborhood,2nd Most Common Neighborhood,3rd Most Common Neighborhood,4th Most Common Neighborhood,5th Most Common Neighborhood,6th Most Common Neighborhood,7th Most Common Neighborhood,8th Most Common Neighborhood,9th Most Common Neighborhood,10th Most Common Neighborhood
58,University Heights,African Restaurant,0,University Heights,Fordham,Woodlawn,Concourse Village,Longwood,Kingsbridge Heights,Kingsbridge,Hunts Point,High Bridge,Edgewater Park
71,Fordham,African Restaurant,0,University Heights,Fordham,Woodlawn,Concourse Village,Longwood,Kingsbridge Heights,Kingsbridge,Hunts Point,High Bridge,Edgewater Park
76,Fordham,Greek Restaurant,0,Fordham,Woodlawn,East Tremont,Melrose,Longwood,Kingsbridge Heights,Kingsbridge,Hunts Point,High Bridge,Edgewater Park


In [63]:
bronx_venues_categroy.loc[bronx_venues_categroy['Cluster Labels'] == 1, bronx_venues_categroy.columns[[1] + list(range(5, bronx_venues_categroy.shape[1]))]]

,Neighborhood,Category,Cluster Labels,1st Most Common Neighborhood,2nd Most Common Neighborhood,3rd Most Common Neighborhood,4th Most Common Neighborhood,5th Most Common Neighborhood,6th Most Common Neighborhood,7th Most Common Neighborhood,8th Most Common Neighborhood,9th Most Common Neighborhood,10th Most Common Neighborhood
122,Parkchester,South Indian Restaurant,1,Parkchester,Woodlawn,East Tremont,Longwood,Kingsbridge Heights,Kingsbridge,Hunts Point,High Bridge,Fordham,Edgewater Park
123,Parkchester,Southern / Soul Food Restaurant,1,Parkchester,Woodlawn,East Tremont,Longwood,Kingsbridge Heights,Kingsbridge,Hunts Point,High Bridge,Fordham,Edgewater Park


In [65]:
bronx_venues_categroy.loc[bronx_venues_categroy['Cluster Labels'] == 2, bronx_venues_categroy.columns[[1] + list(range(5, bronx_venues_categroy.shape[1]))]]

,Neighborhood,Category,Cluster Labels,1st Most Common Neighborhood,2nd Most Common Neighborhood,3rd Most Common Neighborhood,4th Most Common Neighborhood,5th Most Common Neighborhood,6th Most Common Neighborhood,7th Most Common Neighborhood,8th Most Common Neighborhood,9th Most Common Neighborhood,10th Most Common Neighborhood
0,Co-op City,Fast Food Restaurant,2,Westchester Square,Fordham,Mount Eden,Kingsbridge,Morrisania,Concourse Village,Pelham Bay,Allerton,Eastchester,East Tremont
1,Co-op City,Restaurant,2,Kingsbridge Heights,Van Nest,Hunts Point,Norwood,Co-op City,Port Morris,Parkchester,Woodlawn,East Tremont,Kingsbridge
2,Co-op City,Chinese Restaurant,2,Bedford Park,Soundview,University Heights,Kingsbridge Heights,Pelham Parkway,Claremont Village,Pelham Gardens,Allerton,Norwood,Melrose
3,Eastchester,Caribbean Restaurant,2,Eastchester,Williamsbridge,Olinville,Kingsbridge,Unionport,Fordham,Claremont Village,Norwood,Concourse,Parkchester
4,Eastchester,Seafood Restaurant,2,City Island,Morrisania,Kingsbridge,Unionport,Eastchester,Woodlawn,East Tremont,Longwood,Kingsbridge Heights,Hunts Point
...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,Kingsbridge Heights,Mexican Restaurant,2,Bedford Park,Belmont,Kingsbridge,Kingsbridge Heights,Concourse Village,Schuylerville,Pelham Parkway,Melrose,Morrisania,North Riverdale
252,Kingsbridge Heights,Chinese Restaurant,2,Bedford Park,Soundview,University Heights,Kingsbridge Heights,Pelham Parkway,Claremont Village,Pelham Gardens,Allerton,Norwood,Melrose
253,Kingsbridge Heights,Chinese Restaurant,2,Bedford Park,Soundview,University Heights,Kingsbridge Heights,Pelham Parkway,Claremont Village,Pelham Gardens,Allerton,Norwood,Melrose
254,Kingsbridge Heights,Spanish Restaurant,2,Fordham,Mount Eden,Mott Haven,Kingsbridge,Morris Heights,Kingsbridge Heights,Hunts Point,High Bridge,Morris Park,Mount Hope


In [66]:
bronx_venues_categroy.loc[bronx_venues_categroy['Cluster Labels'] == 3, bronx_venues_categroy.columns[[1] + list(range(5, bronx_venues_categroy.shape[1]))]]

,Neighborhood,Category,Cluster Labels,1st Most Common Neighborhood,2nd Most Common Neighborhood,3rd Most Common Neighborhood,4th Most Common Neighborhood,5th Most Common Neighborhood,6th Most Common Neighborhood,7th Most Common Neighborhood,8th Most Common Neighborhood,9th Most Common Neighborhood,10th Most Common Neighborhood
112,Clason Point,South American Restaurant,3,Clason Point,Woodlawn,East Tremont,Melrose,Longwood,Kingsbridge Heights,Kingsbridge,Hunts Point,High Bridge,Fordham


In [67]:
bronx_venues_categroy.loc[bronx_venues_categroy['Cluster Labels'] == 4, bronx_venues_categroy.columns[[1] + list(range(5, bronx_venues_categroy.shape[1]))]]

,Neighborhood,Category,Cluster Labels,1st Most Common Neighborhood,2nd Most Common Neighborhood,3rd Most Common Neighborhood,4th Most Common Neighborhood,5th Most Common Neighborhood,6th Most Common Neighborhood,7th Most Common Neighborhood,8th Most Common Neighborhood,9th Most Common Neighborhood,10th Most Common Neighborhood
22,Woodlawn,Italian Restaurant,4,Belmont,Edgewater Park,Pelham Bay,Pelham Parkway,North Riverdale,Woodlawn,Bronxdale,City Island,Concourse,Concourse Village
38,Pelham Parkway,Italian Restaurant,4,Belmont,Edgewater Park,Pelham Bay,Pelham Parkway,North Riverdale,Woodlawn,Bronxdale,City Island,Concourse,Concourse Village
41,Pelham Parkway,Italian Restaurant,4,Belmont,Edgewater Park,Pelham Bay,Pelham Parkway,North Riverdale,Woodlawn,Bronxdale,City Island,Concourse,Concourse Village
44,City Island,Italian Restaurant,4,Belmont,Edgewater Park,Pelham Bay,Pelham Parkway,North Riverdale,Woodlawn,Bronxdale,City Island,Concourse,Concourse Village
80,Fordham,Italian Restaurant,4,Belmont,Edgewater Park,Pelham Bay,Pelham Parkway,North Riverdale,Woodlawn,Bronxdale,City Island,Concourse,Concourse Village
115,Throgs Neck,Italian Restaurant,4,Belmont,Edgewater Park,Pelham Bay,Pelham Parkway,North Riverdale,Woodlawn,Bronxdale,City Island,Concourse,Concourse Village
135,Morris Park,Italian Restaurant,4,Belmont,Edgewater Park,Pelham Bay,Pelham Parkway,North Riverdale,Woodlawn,Bronxdale,City Island,Concourse,Concourse Village
138,Belmont,Italian Restaurant,4,Belmont,Edgewater Park,Pelham Bay,Pelham Parkway,North Riverdale,Woodlawn,Bronxdale,City Island,Concourse,Concourse Village
139,Belmont,Italian Restaurant,4,Belmont,Edgewater Park,Pelham Bay,Pelham Parkway,North Riverdale,Woodlawn,Bronxdale,City Island,Concourse,Concourse Village
140,Belmont,Italian Restaurant,4,Belmont,Edgewater Park,Pelham Bay,Pelham Parkway,North Riverdale,Woodlawn,Bronxdale,City Island,Concourse,Concourse Village


,Neighborhood,Category,Cluster Labels,1st Most Common Neighborhood,2nd Most Common Neighborhood,3rd Most Common Neighborhood,4th Most Common Neighborhood,5th Most Common Neighborhood,6th Most Common Neighborhood,7th Most Common Neighborhood,8th Most Common Neighborhood,9th Most Common Neighborhood,10th Most Common Neighborhood
